In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
import seaborn as sns
import numpy as np

In [ ]:
def prepare_hume_data(path):
    df_facial = pd.read_csv(path, index_col=0)
    df_facial.drop('start', axis=1, inplace=True)
    df_facial.columns = ['hume_anger', 'hume_disgust', 'hume_fear', 'hume_joy', 'hume_sadness', 'hume_surprise', 'hume_neutral', 'start_patch']
    df_facial['start_patch'] = df_facial['start_patch'].astype('int64')
    df_facial = df_facial.groupby('start_patch').mean().reset_index()
    return df_facial
    

In [ ]:
def merge_text_hume_data(df_hume, df_fer):
    df_merged = pd.merge(df_hume, df_fer, on=['start_patch'], how='inner').drop('start_patch', axis=1)
    return df_merged

In [ ]:
def prepare_fer_data(path):
    df_facial = pd.read_csv(path, index_col=0)
    #df_facial.drop('start', axis=1, inplace=True)
    df_facial.columns = ['fer_anger', 'fer_disgust', 'fer_fear', 'fer_joy', 'fer_sadness', 'fer_surprise', 'fer_neutral', 'start_patch']
    df_facial['start_patch'] = df_facial['start_patch'].astype('int64')
    df_facial = df_facial.groupby('start_patch').mean().reset_index()
    return df_facial
    

In [ ]:
def compare_csv_data(df1_name_hume, df2_name):
    # Read the CSV files into pandas DataFrames
    df1 = df1_name_hume
    df2 = df2_name
    
    # Extract the emotion columns from each DataFrame
    hume_emotions = ['hume_anger', 'hume_disgust', 'hume_fear', 'hume_joy', 'hume_sadness', 'hume_surprise', 'hume_neutral']

    fer_emotions = ['fer_anger', 'fer_disgust', 'fer_fear', 'fer_joy', 'fer_sadness', 'fer_surprise', 'fer_neutral']
    
    # # Iterate over the emotion columns and plot the data for each file
    # for emotion in hume_emotions:
    #     plt.figure()
    #     plt.plot(df1['hume_start_patch'], df1[emotion], label= str('hume'))
    #     plt.xlabel('Timestamp')
    #     plt.ylabel('Score')
    #     plt.title('Comparison of ' + emotion)
    #     plt.legend()
    #     plt.show()

    # for emotion in fer_emotions:
    #     plt.figure()
    #     plt.plot(df2['fer_start_patch'], df2[emotion], label= str('fer'))
    #     plt.xlabel('Timestamp')
    #     plt.ylabel('Score')
    #     plt.title('Comparison of ' + emotion)
    #     plt.legend()
    #     plt.show()

    plt.figure()
    plt.plot(df1['start_patch'], df1['hume_anger'], label= str('hume'))
    plt.plot(df2['start_patch'], df2['fer_anger'], label= str('fer'))
    plt.xlabel('Timestamp')
    plt.ylabel('Score')
    plt.title('Comparison of ' + 'Anger')
    plt.legend()
    plt.show()


In [ ]:
def get_correlations_with_binary(df, binary_feature, continuous_features, corr_func= pearsonr ):
    df = df.dropna()
    correlation_data = []
    for feature in continuous_features:
        corr, p_value = corr_func(df[feature], df[binary_feature])
        correlation_data.append((feature, corr, p_value))
    
    data = correlation_data
    # Extract feature names, correlations, and p-values
    features = [item[0] for item in data]
    correlations = [item[1] for item in data]
    p_values = [item[2] for item in data]

    # Create a figure and axes
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot the correlations as vertical bars
    ax.bar(features, correlations, color='steelblue')

    # Set the y-axis label
    ax.set_ylabel('Correlation')

    # Add horizontal gridlines
    ax.yaxis.grid(True)

    # Set the title
    ax.set_title('Correlation with Binary Variable')

    # Add error bars representing p-values
    ax.errorbar(features, correlations, yerr=np.abs(correlations), fmt='none', color='black', capsize=5, label='p-value')

    # Add p-values and absolute correlation values as text annotations
    for feature, correlation, p_value in zip(features, correlations, p_values):
        ax.text(feature, 0, f'p={p_value:.2f}', ha='center', va='bottom')
        ax.text(feature, correlation, f'{abs(correlation):.2f}', ha='center', va='top')

    # Rotate x-axis labels for better visibility
    plt.xticks(rotation=45, ha='right')

    # Display the plot
    plt.tight_layout()
    plt.show()
    return data

In [ ]:
df_hume_1 = prepare_hume_data('../data/hume/processed/Cuban_vocals_afternoon.csv')

In [ ]:
df_fer_1 = prepare_fer_data('../data/FER_Output/processed/Cuban_vocals_afternoon.csv')

In [ ]:
df_merged = merge_text_hume_data(df_hume_1, df_fer_1)

In [ ]:
df_merged.head()

In [ ]:
df_merged_corr = df_merged.corr().drop(['hume_anger', 'hume_disgust', 'hume_fear', 'hume_joy', 'hume_sadness', 'hume_surprise', 'hume_neutral'], axis=1)
df_merged_corr = df_merged_corr.drop(['fer_anger', 'fer_disgust', 'fer_fear', 'fer_joy', 'fer_sadness', 'fer_surprise', 'fer_neutral'],axis=0)
df_merged_corr

In [ ]:
sns.heatmap(df_merged_corr, annot=True, cmap='coolwarm')

# Customize the plot
plt.title('Correlation Matrix')
plt.show()

In [ ]:
compare_csv_data(df_hume_1, df_fer_1)

In [ ]:
df_hume_2 = prepare_hume_data('../data/hume/processed/Cuban_vocals_morning.csv')

In [ ]:
compare_csv_data(df_hume_2, '../data/FER_Output/processed/Cuban_vocals_morning.csv')

In [ ]:
df_hume_3 = prepare_hume_data('../data/hume/processed/Funk_band_morning.csv')

In [ ]:
compare_csv_data(df_hume_3, '../data/FER_Output/processed/Funk_band_morning.csv')

In [ ]:
df_hume_4 = prepare_hume_data('../data/hume/processed/Funk_rhythm_morning.csv')

In [ ]:
compare_csv_data(df_hume_4, '../data/FER_Output/processed/Funk_rhythm_morning.csv')

#### Validation by comparing the max feature for each time slot of 5min

In [ ]:
def prepare_hume_data(path, var_neutral):
    df_facial = pd.read_csv(path, index_col=0)
    df_facial.drop('start', axis=1, inplace=True)
    df_facial.columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral', 'start_patch']
    df_facial['start_patch'] = df_facial['start_patch'].astype('int64')
    df_facial = df_facial.groupby('start_patch').mean().reset_index()
    if var_neutral == True:
        df_facial = df_facial.drop(columns=['start_patch'])
    else:
        df_facial = df_facial.drop(columns=['start_patch', 'neutral'])
    return df_facial
    

In [ ]:
def prepare_fer_data(path, var_neutral):
    df_facial = pd.read_csv(path, index_col=0)
    #df_facial.drop('start', axis=1, inplace=True)
    df_facial.columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral', 'start_patch']
    df_facial['start_patch'] = df_facial['start_patch'].astype('int64')
    df_facial = df_facial.groupby('start_patch').mean().reset_index()
    if var_neutral == True:
        df_facial = df_facial.drop(columns=['start_patch'])
    else:
        df_facial = df_facial.drop(columns=['start_patch', 'neutral'])
    return df_facial
    

In [ ]:
def aggregate_to_flow_nonflow(df_merged_cleaned, var_neutral):
    flow_emotions = ['joy', 'fear', 'surprise']
    if var_neutral == True:
        non_flow =['neutral', 'sadness', 'anger']
    else:
        non_flow =['disgust', 'sadness', 'anger']
    #df_merged_cleaned = df_merged.drop(['segment_id', 'start', 'end', 'negative'], axis=1)
    df_merged_cleaned= df_merged_cleaned.dropna(axis=0)
    df_merged_cleaned
    df_merged_cleaned['flow_emotions'] = df_merged_cleaned[flow_emotions].mean(axis=1)
    df_merged_cleaned['non_flow_emotions'] = df_merged_cleaned[non_flow].mean(axis=1)

    if var_neutral == True:
        df_merged_cleaned = df_merged_cleaned.drop(columns=['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'])
    else:
        df_merged_cleaned = df_merged_cleaned.drop(columns=['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise'])

    

    return df_merged_cleaned

In [ ]:
def create_dataframe_with_icolumns(num_columns):
    column_names = [f'Column{i}' for i in range(1, num_columns + 1)]
    df_new = pd.DataFrame(columns=column_names)    
    
    return df_new


In [ ]:
def select_max_score_to_array(df_aggregated_to_flow_non_flow, df_scores, num_max_scores):
    scores = []
    for index, row in df_aggregated_to_flow_non_flow.iterrows():
        max_score = row.nlargest(num_max_scores)
        df_scores.loc[index] = max_score.index

    # Iterate over each row in the dataframe
    for _, row in df_scores.iterrows():
        # Convert the row to a list and append it to the row_array
        scores.append(row.tolist())
    

    return scores

In [ ]:
def validate_hume_fer(scores_hume, scores_fer):
    # Check if the length of the arrays is the same
    if len(scores_hume) == len(scores_fer):
        # Initialize a count variable to keep track of matching elements
        count = 0

        # Iterate over the arrays using zip()
        for element1, element2 in zip(scores_hume, scores_fer):
            # Compare the elements from both arrays
            if element1 == element2:
                #print(f"Elements match: {element1}")
                count += 1
            else:
                #print(f"Elements do not match: {element1} and {element2}")
                count+= 0

        # Calculate the percentage of matching elements
        percentage = (count / len(scores_hume)) * 100
        print(f"Percentage of matching elements: {percentage}%")
    else:
        print("Arrays have different lengths.")

    return percentage

In [ ]:
def compare_arrays_and_delete(arr1, arr2):
    if len(arr1) > len(arr2):
        del arr1[len(arr2):]
    elif len(arr2) > len(arr1):
        del arr2[len(arr1):]

In [ ]:
def hume_fer_validation(hume_path, fer_path, num_columns, aggregation, var_neutral):
    df_hume = prepare_hume_data(hume_path, var_neutral)
    df_fer = prepare_fer_data(fer_path, var_neutral)

    if aggregation == True:
        df_hume_agg = aggregate_to_flow_nonflow(df_hume, var_neutral)
        df_fer_agg = aggregate_to_flow_nonflow(df_fer, var_neutral)

        df_hume_scores = create_dataframe_with_icolumns(num_columns)
        df_fer_scores = create_dataframe_with_icolumns(num_columns)

        hume_scores = select_max_score_to_array(df_hume_agg, df_hume_scores, num_columns)
        fer_scores = select_max_score_to_array(df_fer_agg, df_fer_scores, num_columns)

    else:
        df_hume_scores = create_dataframe_with_icolumns(num_columns)
        df_fer_scores = create_dataframe_with_icolumns(num_columns)

        hume_scores = select_max_score_to_array(df_hume, df_hume_scores, num_columns)
        fer_scores = select_max_score_to_array(df_fer, df_fer_scores, num_columns)

    compare_arrays_and_delete(hume_scores, fer_scores)

    overall_score = validate_hume_fer(hume_scores, fer_scores)

    return overall_score

In [ ]:
score_incl_neutral_agg = []
number_max_elements = 1
var_aggregation = True
var_neutral = True

score_incl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_afternoon.csv','../data/FER_Output/processed/Cuban_vocals_afternoon.csv', number_max_elements, var_aggregation, var_neutral ))
score_incl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_morning.csv', '../data/FER_Output/processed/Cuban_vocals_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_incl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Funk_band_afternoon.csv', '../data/FER_Output/processed/Funk_band_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_incl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Funk_band_morning.csv', '../data/FER_Output/processed/Funk_band_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_incl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_afternoon.csv', '../data/FER_Output/processed/Funk_rhythm_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_incl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_morning.csv', '../data/FER_Output/processed/Funk_rhythm_morning.csv', number_max_elements,var_aggregation, var_neutral))

validation = sum(score_incl_neutral_agg)/len(score_incl_neutral_agg)

print(f"Aggregation including neutral: {validation}%")

In [ ]:
score_excl_neutral_agg = []
number_max_elements = 1
var_aggregation = True
var_neutral = False

score_excl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_afternoon.csv','../data/FER_Output/processed/Cuban_vocals_afternoon.csv', number_max_elements, var_aggregation, var_neutral ))
score_excl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_morning.csv', '../data/FER_Output/processed/Cuban_vocals_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Funk_band_afternoon.csv', '../data/FER_Output/processed/Funk_band_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Funk_band_morning.csv', '../data/FER_Output/processed/Funk_band_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_afternoon.csv', '../data/FER_Output/processed/Funk_rhythm_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral_agg.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_morning.csv', '../data/FER_Output/processed/Funk_rhythm_morning.csv', number_max_elements,var_aggregation, var_neutral))

validation = sum(score_excl_neutral_agg)/len(score_excl_neutral_agg)

print(f"Aggregation exluding neutral: {validation}%")

In [ ]:
score_incl_neutral = []
number_max_elements = 1
var_aggregation = False
var_neutral = True

score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_afternoon.csv','../data/FER_Output/processed/Cuban_vocals_afternoon.csv', number_max_elements, var_aggregation, var_neutral ))
score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_morning.csv', '../data/FER_Output/processed/Cuban_vocals_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_afternoon.csv', '../data/FER_Output/processed/Funk_band_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_morning.csv', '../data/FER_Output/processed/Funk_band_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_afternoon.csv', '../data/FER_Output/processed/Funk_rhythm_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_morning.csv', '../data/FER_Output/processed/Funk_rhythm_morning.csv', number_max_elements,var_aggregation, var_neutral))

validation = sum(score_incl_neutral)/len(score_incl_neutral)

print(f"Max Emotion value with neutral: {validation}%")

In [ ]:
score_excl_neutral = []
number_max_elements = 1
var_aggregation = False
var_neutral = False

score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_afternoon.csv','../data/FER_Output/processed/Cuban_vocals_afternoon.csv', number_max_elements, var_aggregation, var_neutral ))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_morning.csv', '../data/FER_Output/processed/Cuban_vocals_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_afternoon.csv', '../data/FER_Output/processed/Funk_band_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_morning.csv', '../data/FER_Output/processed/Funk_band_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_afternoon.csv', '../data/FER_Output/processed/Funk_rhythm_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_morning.csv', '../data/FER_Output/processed/Funk_rhythm_morning.csv', number_max_elements,var_aggregation, var_neutral))

validation = sum(score_excl_neutral)/len(score_excl_neutral)

print(f"Max Emotion value without neutral: {validation}%")

In [ ]:
score_excl_neutral = []
number_max_elements = 1
var_aggregation = False
var_neutral = False

#score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_afternoon.csv','../data/FER_Output/processed/Cuban_vocals_afternoon.csv', number_max_elements, var_aggregation, var_neutral ))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_morning.csv', '../data/FER_Output/processed/Cuban_vocals_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_afternoon.csv', '../data/FER_Output/processed/Funk_band_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_morning.csv', '../data/FER_Output/processed/Funk_band_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_afternoon.csv', '../data/FER_Output/processed/Funk_rhythm_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_morning.csv', '../data/FER_Output/processed/Funk_rhythm_morning.csv', number_max_elements,var_aggregation, var_neutral))

validation = sum(score_excl_neutral)/len(score_excl_neutral)

print(f"Max Emotion value without neutral and without first video: {validation}%")

In [ ]:
score_excl_neutral_wo1 = []
number_max_elements = 1
var_aggregation = False
var_neutral = False

score_excl_neutral_wo1.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_afternoon.csv','../data/FER_Output/processed/Cuban_vocals_afternoon.csv', number_max_elements, var_aggregation, var_neutral ))
score_excl_neutral_wo1.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_morning.csv', '../data/FER_Output/processed/Cuban_vocals_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral_wo1.append(hume_fer_validation('../data/hume/processed/Funk_band_afternoon.csv', '../data/FER_Output/processed/Funk_band_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral_wo1.append(hume_fer_validation('../data/hume/processed/Funk_band_morning.csv', '../data/FER_Output/processed/Funk_band_morning.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral_wo1.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_afternoon.csv', '../data/FER_Output/processed/Funk_rhythm_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
score_excl_neutral_wo1.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_morning.csv', '../data/FER_Output/processed/Funk_rhythm_morning.csv', number_max_elements,var_aggregation, var_neutral))

validation = sum(score_excl_neutral_wo1)/len(score_excl_neutral_wo1)

print(f"Max Emotion value without neutral and without first video: {validation}%")

In [149]:
two_score_excl_neutral = []
number_max_elements = 2
var_aggregation = False
var_neutral = False

two_score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_afternoon.csv','../data/FER_Output/processed/Cuban_vocals_afternoon.csv', number_max_elements, var_aggregation, var_neutral ))
two_score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_morning.csv', '../data/FER_Output/processed/Cuban_vocals_morning.csv', number_max_elements,var_aggregation, var_neutral))
two_score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_afternoon.csv', '../data/FER_Output/processed/Funk_band_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
two_score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_morning.csv', '../data/FER_Output/processed/Funk_band_morning.csv', number_max_elements,var_aggregation, var_neutral))
two_score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_afternoon.csv', '../data/FER_Output/processed/Funk_rhythm_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
two_score_excl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_morning.csv', '../data/FER_Output/processed/Funk_rhythm_morning.csv', number_max_elements,var_aggregation, var_neutral))

validation = sum(two_score_excl_neutral)/len(two_score_excl_neutral)

print(f"Two Max Emotion values without neutral: {validation}%")

Percentage of matching elements: 9.67741935483871%
Percentage of matching elements: 18.75%
Percentage of matching elements: 6.25%
Percentage of matching elements: 3.3333333333333335%
Percentage of matching elements: 0.0%
Percentage of matching elements: 3.8461538461538463%
Two Max Emotion values without neutral: 6.976151089054316%


In [150]:
two_score_incl_neutral = []
number_max_elements = 2
var_aggregation = False
var_neutral = True

two_score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_afternoon.csv','../data/FER_Output/processed/Cuban_vocals_afternoon.csv', number_max_elements, var_aggregation, var_neutral ))
two_score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Cuban_vocals_morning.csv', '../data/FER_Output/processed/Cuban_vocals_morning.csv', number_max_elements,var_aggregation, var_neutral))
two_score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_afternoon.csv', '../data/FER_Output/processed/Funk_band_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
two_score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_band_morning.csv', '../data/FER_Output/processed/Funk_band_morning.csv', number_max_elements,var_aggregation, var_neutral))
two_score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_afternoon.csv', '../data/FER_Output/processed/Funk_rhythm_afternoon.csv', number_max_elements,var_aggregation, var_neutral))
two_score_incl_neutral.append(hume_fer_validation('../data/hume/processed/Funk_rhythm_morning.csv', '../data/FER_Output/processed/Funk_rhythm_morning.csv', number_max_elements,var_aggregation, var_neutral))

validation = sum(two_score_incl_neutral)/len(two_score_incl_neutral)

print(f"Two Max Emotion values with neutral: {validation}%")

Percentage of matching elements: 9.67741935483871%
Percentage of matching elements: 12.5%
Percentage of matching elements: 3.125%
Percentage of matching elements: 6.666666666666667%
Percentage of matching elements: 0.0%
Percentage of matching elements: 3.8461538461538463%
Two Max Emotion values with neutral: 5.96920664460987%


Aufgabe: Nochmal tiefer in die Emotionen reingehen, neutral rauswerfen und für die einzelen max emotionen gucken ob übereinstimmt oder zwei oder drei. Dann Ergebnis nochmal für alle Videos machen und vielleicht auch durchschnitt bilden für Paper